# 🎶 Predicting Lyrics
<hr>
<br>
<b> Group name: <strike>AA</strike> <i>My Machine Learning Romance</i> </b> <br>
- Fiorella Tenorio <br>
- Katja Wittfoth <br>
- Rebecca Reilly <br>
- Victoria Suarez <br>
- Viviana Márquez <br>

<br>
<img src='USF.png', style="width:200px;" align="left">
Friday, December 7th, 2018. <br>
<b> MSDS 621 </b> - Introduction to Machine Learning 

## Goal 
<center>
    <img src="are you smarter.png">
    <big>Predict genre based on lyrics</big><br><br>
    🥊 Supervised <i>vs.</i> Unspervised  <i>vs.</i> Humans 🥊
</center> 

## Workflow Pipeline
<center>
    <img src="workflow2.png" style="height:500px">
</center> 

In [2]:
# Libraries
import warnings
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction import stop_words
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator


import glob
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score


import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve
from sklearn.pipeline import make_pipeline
import seaborn as sns
sns.set()

warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'mlxtend'

In [4]:
# Read data
df = pd.read_csv('../lyrics.csv')

In [5]:
# Clean data
def clean_lyrics(text):
    text = re.sub('\n', ' ', text)  # removes new lines
    text = re.sub('\d', '', text)  # removes numbers
    text = re.sub('\t', '', text)  # removes tabs
    text = re.sub('[^A-Za-z ]', '', text)  # removes special character
    text = re.sub('\[Verse\s*\d*:]', ' ', text)  # remove verse indicators
    text = re.sub('\s+', ' ', text)
    words = text.split(" ")
    words = [w for w in words if len(w) > 2]  # removes a, an, to, at, be, ...
    return ' '.join(words)

df = df[pd.notnull(df['lyrics'])]
df = df[df['genre'] != 'Not Available']
df = df[df['genre'] != 'Other']
df['lyrics_clean'] = df.lyrics.apply(clean_lyrics)

df_clean = df[(df.genre == 'Country') | (df.genre == 'Metal') |
              (df.genre == 'Hip-Hop') | (df.genre == 'Jazz') | (df.genre == 'Electronic')]

df_clean = df_clean.sample(frac=1)  # shuffle

## 📊 1. Get data and pre-process

- 380,000+ lyrics  from MetroLyrics in <a href="https://www.kaggle.com/gyani95/380000-lyrics-from-metrolyrics">Kaggle</a>

#### 🛁 Cleaning 
- Remove: `Null` lyrics, "Not available" genre, punctuation, numbers, symbols, spaces, stop words.
- Shuffle data.
- **Genres**: Country, Metal, Hip-Hop, Jazz, Electronic.

In [4]:
df_clean.head()

,index,song,year,artist,genre,lyrics,lyrics_clean
346876,346876,secret-track,2014,childish-gambino,Hip-Hop,No matter what you say or what you do\nWhen I'...,"matter what you say what you When I'm alone, I..."
286644,286644,futuristic-space-doubleswee-rap-in-the-galaxy,2014,doubleswee,Hip-Hop,"You, Doubleswee\nIn space\nWhat up guys\nYeah ...","You, Doubleswee space What guys Yeah it's Chil..."
166706,166706,just-one-of-those-things,2014,django-reinhardt,Jazz,It was just one of those things\nJust one of t...,was just one those things Just one those crazy...
336273,336273,gimme-half,2009,the-devil-wears-prada,Metal,"Keep this on your mind,\nKeep it within your e...","Keep this your mind, Keep within your eyelids...."
53140,53140,horus-rises,2010,at-the-throne-of-judgement,Metal,"Rise.\nA sentence given, to the afflicted livi...","Rise. sentence given, the afflicted living, it..."


## 👠 2. Modeling

### ✂️ Splitting data

In [7]:
train, test = train_test_split(df_clean, test_size=0.2)
train_lyrics = train['lyrics']
train_genre = train['genre']
test_lyrics = test['lyrics']
test_genre = test['genre']

### 🥊 Supervised model: Multinomial Naive-Bayes

In [12]:
pipe_lr_cv = Pipeline([
    ('cv', CountVectorizer(stop_words='english',
                             lowercase=False,
                             analyzer='word')),
    ('lr', LogisticRegression())
])

pipe_lr_tf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english',
                             lowercase=False,
                             analyzer='word')),
    ('lr', LogisticRegression())
])

pipe_nb_cv = Pipeline([
    ('cv', CountVectorizer(stop_words='english',
                             lowercase=False,
                             analyzer='word')),
    ('nb', MultinomialNB())
])

pipe_nb_tf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english',
                             lowercase=False,
                             analyzer='word')),
    ('nb', MultinomialNB())
])

pipe_svc_cv = Pipeline([
    ('cv', CountVectorizer(stop_words='english',
                             lowercase=False,
                             analyzer='word')),
    ('svc', LinearSVC())
])

pipe_svc_tf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english',
                             lowercase=False,
                             analyzer='word')),
    ('svc', LinearSVC())
])

pipelines = [pipe_lr_cv, pipe_lr_tf, pipe_nb_cv, pipe_nb_tf, pipe_svc_cv, pipe_svc_tf]

In [13]:
# Fit pipe
for pipe in pipelines:
    pipe.fit(train_lyrics, train_genre)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [14]:
methods = []

for pipe in pipelines:
    name = pipe.steps[-1][1].__class__.__name__.split('.')[-1]
    vect = pipe.steps[0][1].__class__.__name__.split('.')[-1]

    predicted = pipe.predict(test_lyrics)
    accuracy = accuracy_score(predicted, test['genre'])
    conf_mat = confusion_matrix(test['genre'], predicted)

    methods.append([name, vect, accuracy, conf_mat])

methods = pd.DataFrame(methods)
methods.columns = ['Model', 'Vectorizer', 'Accuracy', "Confussion_Matrix"]
methods["Strategy"] = methods["Model"] + " -- " + methods["Vectorizer"]
methods.set_index(["Strategy"], inplace=True)

In [15]:
methods

,Model,Vectorizer,Accuracy,Confussion_Matrix
Strategy,,,,
LogisticRegression -- CountVectorizer,LogisticRegression,CountVectorizer,0.773104,"[[2360, 120, 86, 217, 174], [164, 581, 220, 10..."
LogisticRegression -- TfidfVectorizer,LogisticRegression,TfidfVectorizer,0.770254,"[[2411, 60, 110, 154, 222], [198, 421, 287, 68..."
MultinomialNB -- CountVectorizer,MultinomialNB,CountVectorizer,0.728828,"[[2676, 23, 76, 52, 130], [480, 359, 274, 45, ..."
MultinomialNB -- TfidfVectorizer,MultinomialNB,TfidfVectorizer,0.624691,"[[1136, 9, 1373, 2, 437], [42, 51, 910, 4, 555..."
LinearSVC -- CountVectorizer,LinearSVC,CountVectorizer,0.734275,"[[2165, 151, 139, 339, 163], [172, 681, 218, 1..."
LinearSVC -- TfidfVectorizer,LinearSVC,TfidfVectorizer,0.786026,"[[2396, 101, 94, 217, 149], [168, 649, 242, 10..."


## we know LinearSVC -- TfidfVectorizer is the best
### go back and manual grid search

In [38]:
pipe_svc_tf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english',
                             lowercase=False,
                             analyzer='word')),
    ('svc', LinearSVC())])

In [43]:
grid = {'svc__C': np.arange(0.01,100,10)}

gs = GridSearchCV(pipe_svc_tf,
                  param_grid=grid,
                  scoring='accuracy')

# loss : string, ‘hinge’ or ‘squared_hinge’ 
# penalty : string, ‘l1’ or ‘l2’

In [ ]:
gs.fit(train_lyrics, train_genre)

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [ ]:
gs.best_estimator_.steps[0][1]  # best model

In [ ]:
pred = gs.best_estimator_.predict(test_lyrics)

# Recap ✍️